In [5]:
#!pip install sktime
#!pip install sktime[all_extras]

In [1]:
from sktime.classification.hybrid import HIVECOTEV1

In [2]:
from sktime.datasets import load_from_tsfile_to_dataframe as load_ts

In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

In [4]:
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.model_selection import KFold

In [5]:
X5_test, y5_test = load_ts("ECGFiveDays/ECGFiveDays_TEST.ts")
X5_small, y5_small = load_ts("ECGFiveDays/ECGFiveDays_TRAIN.ts")
X200_train, y200_train = load_ts("ECG200/ECG200_TRAIN.ts")
X5000_train, y5000_train = load_ts("ECG5000/ECG5000_TRAIN.ts")
X200_test, y200_test = load_ts("ECG200/ECG200_TEST.ts")
X5000_test, y5000_test = load_ts("ECG5000/ECG5000_TEST.ts")

In [6]:
df = pd.read_csv('mitbih_train.csv')

In [7]:
df = pd.concat([df.iloc[500:7600],
           df.iloc[73066:73236],
           df.iloc[75222:75995],
           df.iloc[80490:80599],
           df.iloc[81150:81999]])
X_ECG = np.array(df.iloc[:, :-1])
print(len(X_ECG))
y_ECG = np.array(df.iloc[ :,-1])
print(len(y_ECG))

9001
9001


In [8]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from models import TimeSeriesCnnModel
from models import HIVECOTEV1_MY

from utils import TSCDataset
from utils import TSCDatasetNN
from utils import evaluate
from utils import train_loop

from torch.optim.lr_scheduler import ReduceLROnPlateau

In [9]:
hc = HIVECOTEV1_MY()

In [19]:
hc.fit(X_ECG, y_ECG)

STC:   -8509.56810760498
TSF:   -1654.7263369560242
RISE:   -10113.727526187897
CBOSS:   -2015.0057821273804


HIVECOTEV1_MY()

In [20]:
hc.save("./HC_MY_BIG_ECG9_5classes")

<zipfile.ZipFile filename='HC_MY_BIG_ECG9_5classes.zip' mode='r'>

In [10]:
hc = hc.load_from_path("/home/ironic/repos/TSC_NN_Experiments/HC_MY_BIG_ECG9_5classes.zip")

In [ ]:
predictions = hc.predict(X_ECG)

In [12]:
predictions = hc.predict(X_ECG)
print("PRECISION: ", metrics.precision_score(list(map(lambda x: int(x), y_ECG)), predictions, average='macro'))
print("RECALL: ",metrics.recall_score(list(map(lambda x: int(x), y_ECG)), predictions, average='macro'))
print("F1: ", metrics.f1_score(list(map(lambda x: int(x), y_ECG)), predictions, average='macro'))

NameError: name 'y_ECG' is not defined

In [8]:
df_test = pd.read_csv('mitbih_test.csv')
X_ECG_test = np.array(df_test.iloc[:, :-1])
print(len(X_ECG_test))
y_ECG_test = np.array(df_test.iloc[:,-1])
print(len(y_ECG_test))

21891
21891


In [19]:
predictions = hc.predict(X_ECG_test)

In [ ]:
print("PRECISION: ", metrics.precision_score(list(map(lambda x: int(x), y_ECG_test)), predictions, average='macro'))
print("RECALL: ",metrics.recall_score(list(map(lambda x: int(x), y_ECG_test)), predictions, average='macro'))
print("F1: ", metrics.f1_score(list(map(lambda x: int(x), y_ECG_test)), predictions, average='macro'))

In [6]:
hc1 = HIVECOTEV1()
hc1.fit(X5_small, np.asarray(list(map(lambda x: int(x), y5_small))))
predictions = hc1.predict(X5_test)
print("PRECISION: ", metrics.precision_score(list(map(lambda x: int(x), y5_test)), predictions))
print("RECALL: ",metrics.recall_score(list(map(lambda x: int(x), y5_test)), predictions))
print("F1: ", metrics.f1_score(list(map(lambda x: int(x), y5_test)), predictions))

PRECISION:  1.0
RECALL:  1.0
F1:  1.0


In [8]:
hc2 = HIVECOTEV1()
hc2.fit(X200_train, np.asarray(list(map(lambda x: int(x), y200_train))))
predictions = hc2.predict(X200_test)
print("PRECISION: ", metrics.precision_score(list(map(lambda x: int(x), y200_test)), predictions))
print("RECALL: ",metrics.recall_score(list(map(lambda x: int(x), y200_test)), predictions))
print("F1: ", metrics.f1_score(list(map(lambda x: int(x), y200_test)), predictions))

PRECISION:  0.8695652173913043
RECALL:  0.9375
F1:  0.9022556390977444


In [10]:
predictions

array([1, 1, 1, ..., 2, 2, 4])

In [17]:
print("PRECISION: ", metrics.precision_score(list(map(lambda x: int(x), y5000_test)), predictions, average='micro'))
print("RECALL: ",metrics.recall_score(list(map(lambda x: int(x), y5000_test)), predictions, average='micro'))
print("F1: ", metrics.f1_score(list(map(lambda x: int(x), y5000_test)), predictions, average='micro'))

PRECISION:  0.9451111111111111
RECALL:  0.9451111111111111
F1:  0.9451111111111111


In [ ]:
list(map(lambda x: int(x), y5_small))

[1, 1, 2, 2, 1, 2, 2, 1, 2, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 2, 1]

In [16]:
tsc = TimeSeriesCnnModel()

In [14]:
df = pd.read_csv('mitbih_train.csv')
X_ECG = df.iloc[:, :-1]
print(len(X_ECG))
y_ECG = df.iloc[: ,-1]
print(len(y_ECG))

87553
87553


In [11]:
df = pd.read_csv('mitbih_test.csv')
X_ECG_test = df.iloc[:, :-1]
print(len(X_ECG_test))
y_ECG_test = df.iloc[: ,-1]
print(len(y_ECG_test))

21891
21891


In [12]:
dataset_train = TSCDatasetNN(X_ECG, y_ECG)
dataset_test = TSCDataset(X_ECG_test, y_ECG_test)

In [13]:
batch_size = 128
train_dataloader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)

In [17]:
optimizer = torch.optim.Adam(tsc.parameters(), lr=0.001)
# scheduler = ReduceLROnPlateau(optimizer, mode='min',
#     factor=0.1, patience=10, threshold=0.0001, threshold_mode='abs')
loss_fn = torch.nn.CrossEntropyLoss()
train_loop(tsc, 15, optimizer, loss_fn, train_dataloader, test_dataloader, len(dataset_train)//batch_size)

EPOCH:  0


  0%|          | 0/70 [00:00<?, ?it/s]

/home/ironic/repos/TSC_NN_Experiments/utils.py:49: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  return torch.tensor([self.x[i]]).float(), map_to_scores(float(self.y[i]))


train loss: 1.604694
train loss: 0.995865
train loss: 0.680895
train loss: 0.564389
train loss: 0.709929
train loss: 0.660659
train loss: 0.770883
train loss: 0.689747
RECALL:  0.2
F1 SCORE:  0.18113377324535093
EPOCH:  1


  0%|          | 0/70 [00:00<?, ?it/s]

...

train loss: 0.162621
train loss: 0.117075
train loss: 0.092723
train loss: 0.130949
train loss: 0.166021
train loss: 0.037621
train loss: 0.047351
train loss: 0.047936
RECALL:  0.7789302769343315
F1 SCORE:  0.8230132406697959
EPOCH:  9


  0%|          | 0/70 [00:00<?, ?it/s]

train loss: 0.063300
train loss: 0.110770
train loss: 0.022399
train loss: 0.099975
train loss: 0.164546
train loss: 0.115836
train loss: 0.087555
train loss: 0.099471
RECALL:  0.8014068644798169
F1 SCORE:  0.822757490794972
EPOCH:  10


  0%|          | 0/70 [00:00<?, ?it/s]

train loss: 0.047945
train loss: 0.128287
train loss: 0.082416
train loss: 0.070994
train loss: 0.028155
train loss: 0.028756
train loss: 0.118599
train loss: 0.171775
RECALL:  0.7777744539907591
F1 SCORE:  0.8212482097281348
EARLY STOPPING


In [18]:
optimizer = torch.optim.Adam(tsc.parameters(), lr=0.0001)
# scheduler = ReduceLROnPlateau(optimizer, mode='min',
#     factor=0.1, patience=10, threshold=0.0001, threshold_mode='abs')
loss_fn = torch.nn.CrossEntropyLoss()
train_loop(tsc, 10, optimizer, loss_fn, train_dataloader, test_dataloader, len(dataset_train)//batch_size)

EPOCH:  0


  0%|          | 0/70 [00:00<?, ?it/s]

train loss: 0.072850
train loss: 0.036676
train loss: 0.010652
train loss: 0.064914
train loss: 0.026749
train loss: 0.052966
train loss: 0.040487
train loss: 0.007951
RECALL:  0.8154183418697116
F1 SCORE:  0.844887423556316
EPOCH:  1


  0%|          | 0/70 [00:00<?, ?it/s]

train loss: 0.049827
train loss: 0.008565
train loss: 0.048667
train loss: 0.069049
train loss: 0.024071
train loss: 0.034142
train loss: 0.045864
train loss: 0.036866
RECALL:  0.8340080105928939
F1 SCORE:  0.8466530850484343
EPOCH:  2


  0%|          | 0/70 [00:00<?, ?it/s]

train loss: 0.021505
train loss: 0.024140
train loss: 0.015497
train loss: 0.062082
train loss: 0.044435
train loss: 0.055927
train loss: 0.047844
train loss: 0.088131
RECALL:  0.8260785700982922
F1 SCORE:  0.8459015590295152
EARLY STOPPING


In [19]:
optimizer = torch.optim.Adam(tsc.parameters(), lr=0.00001)
# scheduler = ReduceLROnPlateau(optimizer, mode='min',
#     factor=0.1, patience=10, threshold=0.0001, threshold_mode='abs')
loss_fn = torch.nn.CrossEntropyLoss()
train_loop(tsc, 10, optimizer, loss_fn, train_dataloader, test_dataloader, len(dataset_train)//batch_size)

EPOCH:  0


  0%|          | 0/70 [00:00<?, ?it/s]

train loss: 0.030772
train loss: 0.021049
train loss: 0.011782
train loss: 0.064259
train loss: 0.008073
train loss: 0.048861
train loss: 0.011039
train loss: 0.078447
RECALL:  0.8249607943332758
F1 SCORE:  0.8491166022644532
EPOCH:  1


  0%|          | 0/70 [00:00<?, ?it/s]

train loss: 0.066214
train loss: 0.021027
train loss: 0.032161
train loss: 0.011170
train loss: 0.023051
train loss: 0.082149
train loss: 0.009212
train loss: 0.015303
RECALL:  0.8253618796820529
F1 SCORE:  0.8473063594998553
EPOCH:  2


  0%|          | 0/70 [00:00<?, ?it/s]

train loss: 0.031675
train loss: 0.044138
train loss: 0.042277
train loss: 0.018332
train loss: 0.011040
train loss: 0.017985
train loss: 0.006473
train loss: 0.025536
RECALL:  0.8249806331070738
F1 SCORE:  0.8479521716292121
EARLY STOPPING


In [20]:
rec, f1 = evaluate(tsc, test_dataloader)
hc.cnn_weight_= (rec + f1) / 2
hc.cnn_weight_

0.836466402368143

In [21]:
hc.cnn = tsc

In [25]:
#np.array(X_ECG_test.iloc[0:2, :])

In [26]:
# X=np.array(X_ECG_test.iloc[0:5, :])
# dataset_MY = TSCDataset_MY(X)
# dataloader_MY = DataLoader(dataset_MY, batch_size=dataset_MY.__len__(), shuffle=False)
# softmax = nn.Softmax()
# dists = np.zeros((X.shape[0], hc.n_classes_))
# for _, X_MY in enumerate(dataloader_MY):
#     cnn_pred = softmax(hc.cnn(X_MY)).detach().numpy()
#     dists = np.add(
#     dists,
#     cnn_pred
#     * (np.ones(hc.n_classes_) * hc.cnn_weight_),)

In [22]:
predictions_cnn = hc._predict_cnn(np.array(X_ECG_test))

/home/ironic/repos/TSC_NN_Experiments/models.py:344: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  cnn_pred = softmax(self.cnn(X_MY)).detach().numpy()


In [23]:
predictions= hc.predict(np.array(X_ECG_test))

In [24]:
predictions_cnn

array([0., 0., 0., ..., 4., 4., 4.])

In [25]:
print("PRECISION: ", metrics.precision_score(list(map(lambda x: int(x), y_ECG_test)), predictions, average='macro'))
print("RECALL: ",metrics.recall_score(list(map(lambda x: int(x), y_ECG_test)), predictions, average='macro'))
print("F1: ", metrics.f1_score(list(map(lambda x: int(x), y_ECG_test)), predictions, average='macro'))

PRECISION:  0.9348306817199301
RECALL:  0.7518105758307962
F1:  0.8219176448603877


In [26]:
print("PRECISION: ", metrics.precision_score(list(map(lambda x: int(x), y_ECG_test)), predictions_cnn, average='macro'))
print("RECALL: ",metrics.recall_score(list(map(lambda x: int(x), y_ECG_test)), predictions_cnn, average='macro'))
print("F1: ", metrics.f1_score(list(map(lambda x: int(x), y_ECG_test)), predictions_cnn, average='macro'))

PRECISION:  0.930367154706271
RECALL:  0.784696207134058
F1:  0.8423425553188004
